# 🧠 Ejercicio 3.3: Memoria Compartida

**Módulo 3: Agentes Autónomos**  
**Tiempo estimado**: 10 minutos  
**Ejercicio**: 3.3

---

## 🎯 Objetivos de Aprendizaje

1. ✅ Usar operator.add para acumular
2. ✅ Compartir hechos entre agentes
3. ✅ Contexto persistente
4. ✅ Multi-turno conversations


In [ ]:
# ╔════════════════════════════════════════════════════════════╗
# ║  📋 SETUP Y VERIFICACIÓN                                  ║
# ╚════════════════════════════════════════════════════════════╝

%pip install -q langgraph langchain-openai python-dotenv

import os, sys
from dotenv import load_dotenv
sys.path.append(os.path.abspath('../..'))
load_dotenv()

print('='*50)
print('   SETUP VERIFICATION')
print('='*50)
print(f"✅ Python {sys.version.split()[0]}")
print(f"{'✅' if os.getenv('OPENAI_API_KEY') else '❌'} OpenAI API Key")
print(f"\n🎬 Ready!\n")

## 💬 SCRIPT

> "Memoria compartida: Los agentes RECUERDAN.
> No empiezan de cero cada vez."

In [ ]:
from typing import Annotated
import operator
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import StateGraph, START, END

class MemoryState(TypedDict):
    messages: Annotated[list, operator.add]
    facts: Annotated[list, operator.add]  # ← Acumula

llm = ChatOpenAI(model='gpt-4o-mini')
print('✅ Setup')

In [ ]:
# Agente con memoria

def agent_with_memory(state):
    '''Agente que usa y actualiza memoria'''
    messages = state['messages']
    facts = state.get('facts', [])
    
    # Construir contexto con memoria
    context = SystemMessage(content=f"Hechos conocidos: {', '.join(facts)}")
    full_messages = [context] + messages
    
    print(f'🧠 Memoria actual: {facts}')
    
    response = llm.invoke(full_messages)
    
    # Extraer nuevos hechos (simplificado)
    new_facts = []
    content_lower = response.content.lower()
    if 'nombre' in content_lower and 'es' in content_lower:
        # Extrae hechos simples
        new_facts.append(f"Nombre mencionado en conversación")
    
    return {
        'messages': [response],
        'facts': new_facts  # Se AGREGAN a los existentes
    }

print('✅ Agente con memoria definido')

In [ ]:
# Construir grafo simple

graph = StateGraph(MemoryState)
graph.add_node('agent', agent_with_memory)
graph.add_edge(START, 'agent')
graph.add_edge('agent', END)

app = graph.compile()
print('✅ Sistema con memoria compilado')

In [ ]:
# 🎬 DEMO: Conversación multi-turno

conversacion = [
    'Me llamo Juan',
    '¿Cuál es mi nombre?',
    'Vivo en México',
    '¿Dónde vivo?'
]

state = {'messages': [], 'facts': []}

print('='*60)
print('🧠 MEMORIA COMPARTIDA')
print('='*60)

for turno in conversacion:
    print(f'\nUsuario: {turno}')
    state['messages'] = [HumanMessage(turno)]
    result = app.invoke(state)
    
    # Actualizar state con resultados acumulados
    state['facts'] = result['facts']
    
    print(f'Agente: {result["messages"][-1].content}')
    print(f'Memoria: {result["facts"]}')

---

## ✅ CHECKPOINT FINAL

- [ ] Código ejecuta sin errores
- [ ] Conceptos clave entendidos
- [ ] Listos para continuar

### 💬 PREGUNTA:
> "¿Alguna duda antes de continuar?"
